### There are two entities: **Alice and Bob,** to do five main steps.

### **FIVE main steps**

1. **Alice** <u>creates </u>the required parameters for DH key exchange, creates her own private and public keys, and sends the public parameters and her public keys to Bob
2. **Bob** creates his own public key, and sends it to Alice. Bob also creates a derived shared key \(to be used later for AES\-128 encryption\)
3. **Alice** received Bob's public key and also creates a derived shared key \(to be used later for AES\-128 encryption\)
4. **Bob** <u>reads </u>the content of message.txt and <u>sends </u>the following to **Alice**:
   - The content of message.txt encrypted using the shared key \(generated in step 2\)
   - The \(SHA256\) hash value of the message
5. **Alice** <u>receives </u>the set of information from **Bob**, and <u>does </u>the following:
   1. Decrypt the hash value of the message, 
   2. Decrypt the message using the shared key \(generated in step 3\)
   3. Hash \(the received and decrypted\) message and checks which it is identical to \(the received\) hash value.

**Your task is to complete the below code to implement the above task.**



## Step 1: Alice generates DH parameters



In [110]:
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [111]:
def step_1():
    parameters = dh.generate_parameters(generator=2, key_size=512)
    p = parameters.parameter_numbers().p
    g = parameters.parameter_numbers().g
    alice_private_key = parameters.generate_private_key()
    alice_y = alice_private_key.public_key().public_numbers().y
    alice_x = alice_private_key.private_numbers().x
    return alice_x, alice_y, p, g

In [112]:
alice_x, alice_y, p, g = step_1()
print(alice_x)
print(alice_y)
print(p)
print(g)

6605432472700418871328795393237070000487980887913601113648074531806349780600511609372842693511828371720153458499815659515496175394951903831584971385724260
9910018089555469257847513958647911550183003510429885664587344403804419983458091537834452066174328832129204264430693402451342214195574713061081241489636904
11734945226824123671272289197759227603883293175729238309546396106746879905683757482186930427155557676210144531750470926047659614626853488999810832536511819
2


## Step 2: Bob generates DH parameters



In [113]:
def step_2(p, g, alice_y):
    pn = dh.DHParameterNumbers(p, g)
    parameters = pn.parameters()
    bob_private_key = parameters.generate_private_key()
    bob_y = bob_private_key.public_key().public_numbers().y
    bob_x = bob_private_key.private_numbers().x

    alice_public_number = dh.DHPublicNumbers(alice_y, pn)
    alice_public_key = alice_public_number.public_key()

    bob_shared_key = bob_private_key.exchange(alice_public_key)
    bob_derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(bob_shared_key)
    return bob_derived_key, bob_y


bob_derived_key, bob_y = step_2(p, g, alice_y)
print(bob_derived_key)
print(bob_y)

b'\xce\xb0\xc3\x90\xbe\xa0\xe3\xfb\xb5"\xb8w\xa1\xc6\x13\xf57\xdc\xc1\x84\x98M\x87qA\x0b\xff\n6\xb3\x8e\xb3'
8353388745652751323984355860261986294060526853738873067579911631438351142858066745502561989884430862582379407065769810258473148305504423257382075901711814


## Step 3:  Alice generates final key



In [114]:
def step_3(alice_x, p, g, alice_y, bob_y):
    pn = dh.DHParameterNumbers(p, g)
    bob_public_number = dh.DHPublicNumbers(bob_y, pn)
    bob_public_key = bob_public_number.public_key()

    alice_public_number = dh.DHPublicNumbers(alice_y, pn)
    alice_private_key = dh.DHPrivateNumbers(alice_x,
                                            alice_public_number).private_key()
    alice_shared_key = alice_private_key.exchange(bob_public_key)

    alice_derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(alice_shared_key)
    return alice_derived_key


alice_derived_key = step_3(alice_x, p, g, alice_y, bob_y)

print(alice_derived_key)
print(bob_derived_key == alice_derived_key)

b'\xce\xb0\xc3\x90\xbe\xa0\xe3\xfb\xb5"\xb8w\xa1\xc6\x13\xf57\xdc\xc1\x84\x98M\x87qA\x0b\xff\n6\xb3\x8e\xb3'
True


## Step 4: Bob sends a message to Alice



In [115]:
from hashlib import sha256
from Crypto.Cipher import AES
import secrets

BLOCK_SIZE = 16  # for AES 128
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * chr(BLOCK_SIZE - len(
    s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]


def readFileContent(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()


def hashSHA256(fileContent):
    return sha256(fileContent.encode()).hexdigest()


def encryptTheContentWithAESCBC(fileContent, key):
    iv = secrets.token_bytes(BLOCK_SIZE)
    enc = AES.new(key, AES.MODE_CBC, iv)
    data = pad(fileContent).encode("utf-8")
    cipherText = enc.encrypt(data)
    cipherTextHex = iv.hex() + cipherText.hex()
    return cipherTextHex

In [116]:
def step_4(bob_derived_key):
    fileContent = readFileContent("message.txt")
    original_message_hash = hashSHA256(fileContent)
    cipher = encryptTheContentWithAESCBC(fileContent, bob_derived_key)
    return cipher, original_message_hash


cipher, original_message_hash = step_4(bob_derived_key)
print(cipher)
print(original_message_hash)

21b4825a0a935bb969ff908d850d774a474f0c1f31eb3f581a2e7dfb8baa4a1897c19ba907935bf8a39339026ff18a0e874723f0430014a0dd9447156eb045a4
6f3ad1eeab875b5a0e52720bd6225556824431b1bcfd01e3f06964316225e86a


## Step 5: Alice receives the message from Bob



In [117]:
def decryptAES(cipherText, alice_derived_key):
    iv = bytes.fromhex(cipherText[:32])
    cipherText = bytes.fromhex(cipherText[32:])
    dec = AES.new(alice_derived_key, AES.MODE_CBC, iv)
    return unpad(dec.decrypt(cipherText)).decode('utf-8')

In [118]:
def step_5(alice_derived_key, cipher, original_message_hash):
    decrypted_message = decryptAES(cipher, alice_derived_key)
    result = hashSHA256(decrypted_message) == original_message_hash
    return decrypted_message, result  # decrypted_message is the decrypted cipher, and result is a Boolean True/False variable.

In [119]:
print(step_5(alice_derived_key, cipher, original_message_hash)) # The output should be the message along with the verification result (true in this case)

('this is a secret message from Bob to Alice', True)
